In [ ]:
from __future__ import print_function
import pandas as pd
import numpy as np
import csv
import os
import torch

import random

from PIL import Image
from skimage import io

verbose = False

In [ ]:
class mini_batch(torch.nn.Module):
    def __init__(self, dat_folder,transform,percent):
        super(mini_batch, self).__init__()
        self.root_dir = os.path.dirname(dat_folder) 
        self.transform = transform
        self.percent = percent

        #list of folders 
        ind = [individual for individual in os.listdir(self.root_dir) if os.path.isdir(os.path.join(self.root_dir,individual))]
            
        self.ind = ind        
        
    def loadfile(self, data_file):
        image = io.imread(data_file)
        if len(image.shape)<3:
            image = np.stack((image,)*3, axis=-1)
        return image
    
    
    def prepare_batch(self):

        train_line_list = []
        test_line_list = []
        
        labels_train = []
        labels_test = []

        for i,individual in enumerate(self.ind):
            folder_path = os.path.join(self.root_dir,individual)
            name_images = [name for name in os.listdir(folder_path)]  

            random.shuffle(name_images)
            N = len(name_images)

            for j in range(int(N*self.percent)):
                if os.path.isfile(os.path.join(folder_path,name_images[j])):
                    image_filename = os.path.join(individual,name_images[j])
                    if verbose: print("image {}{}".format(j,image_filename))
                    train_line_list.append(image_filename)
                    labels_train.append(i)
 

            for j in range(int(N*self.percent),N):
                if os.path.isfile(os.path.join(folder_path,name_images[j])):
                    image_filename = os.path.join(individual,name_images[j])
                    if verbose: print("image {}{}".format(j,image_filename))
                    test_line_list.append(image_filename)
                    labels_test.append(i)

        image_train = [self.transform(Image.fromarray(self.loadfile(os.path.join(self.root_dir,train_line_list[i])))) for i in range(len(train_line_list))]                    
        image_test = [self.transform(Image.fromarray(self.loadfile(os.path.join(self.root_dir,test_line_list[i])))) for i in range(len(test_line_list))]                    
        return image_train, labels_train, image_test, labels_test
    
    